# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [ ]:
!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions import MultivariateNormal
import torch.distributions.normal as dist

In [ ]:
# select this option to load version 1 (with a single agent) of the environment
env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
torch.manual_seed(0) # set random seed
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
from torchvision import transforms

env_info = env.reset(train_mode=True)[brain_name]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
states = env_info.vector_observations                  # get the current state (for each agent)
#print(states)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
sample_states = []
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1    
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    #print('action:',actions,'next states',next_states,'reward:',rewards)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    sample_states.append(states)
    if np.any(dones):                                  # exit loop if episode finished
        break
    print('a state=',states)
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [ ]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]

In [ ]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 0.5#1. / np.sqrt(fan_in)
    return (-lim, lim)

class Policy(nn.Module):
    def __init__(self, s_size=33, h_size=256,h2_size=128, a_size=4):
        super(Policy, self).__init__()
        #self.fc1 = nn.Linear(s_size, h_size)
        #self.fc2 = nn.Linear(h_size, a_size)
        #self.bn1 = nn.BatchNorm1d(num_features=s_size)
        
        self.transform = transform
        self.fc1 = nn.Linear(s_size, h_size)
        self.fc2 = nn.Linear(h_size, h2_size)
        self.fc3 = nn.Linear(h2_size, a_size)
        #self.fc3 = nn.Linear(fc2_units, action_size)
        #self.bn1 = nn.BatchNorm1d(1, s_size)
        self.tanh = nn.Tanh()
        self.L =  0.1*torch.randn(a_size,a_size).to(device)
        #self.L =  torch.randn(a_size,a_size,device=device)  
        #self.std = nn.Parameter(0.2*torch.eye(a_size,a_size))
        self.std = nn.Parameter(torch.randn(a_size))

        #self.reset_parameters()
        
    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)  
        
        
    def forward(self, x):
        # make sure your x is normalized  
        #print('x.shape',x.shape)
        
        
        #x = self.bn1(x)
        x = self.fc1(x)        
        x = self.fc2(x) 
        xout = self.fc3(x).squeeze(0)
        #m = dist.Normal(xout, self.std)
        xout = self.tanh(xout)#F.softmax(x.squeeze(0))
        #print(xout.shape)
        return xout
    
    def act(self, state):
        #state = torch.from_numpy(state).float().unsqueeze(0).to(device_cuda)
        fw = self.forward(state)
        ####### This ########
        #m = dist.Normal(fw, torch.abs(self.std))
        #print(self.L)
        std = torch.abs(self.L) @ torch.abs(self.L.transpose(0, 1))
        m = MultivariateNormal(fw,std)
        #print('std',self.std)
        #actions = torch.clamp(m.sample(),-1,1)
        #return actions.squeeze(), m.log_prob(actions.squeeze())
        #return actions,m.log_prob(actions).sum()
        
        return fw, m.log_prob(fw)#.sum(-1)/4.0
        ######## or This
        #actions =F.tanh(fw)
        #probs1 = self.sig(fw)
        #return actions, probs1.sum()/4.0

In [ ]:
class Normalizer():
    def __init__(self, num_inputs):
        self.n = torch.zeros(num_inputs).float().to(device)
        self.mean = torch.zeros(num_inputs).float().to(device)
        self.mean_diff = torch.zeros(num_inputs).float().to(device)
        self.var = torch.zeros(num_inputs).float().to(device)

    def observe(self, x):
        self.n += x.shape[0]
        last_mean = self.mean.clone()
        #print('x.shape',x.shape)
        self.mean += (x-self.mean).sum(0)/self.n
        
        
        self.mean_diff += ((x-last_mean)*(x-self.mean)).sum(0)
        self.var = torch.clamp(self.mean_diff/self.n, min=1e-2)

    def normalize(self, inputs):
        obs_std = torch.sqrt(self.var)
        #print('wtf mean',self.mean)#/self.n.expand_as(x)
        out = (inputs - self.mean)/obs_std
        
        return out

In [ ]:
a = np.array([1,2,3])
b = np.array([4,5,6])

print(sum(a*b))
p = [a_*b_ for a_,b_ in zip(a,b)]
print(p)

In [ ]:
state_mean = torch.from_numpy(np.mean(sample_states,axis=0)).float().to(device)
state_std = torch.from_numpy(np.mean(np.std(sample_states,axis=0),axis=0)).float().to(device)
print('state_mean',state_mean.shape,'state_std',type(state_std),'len',len(state_mean))
transform =  Normalizer(state_mean.shape[1])
policy = Policy().to(device)
optimizer = optim.Adam(policy.parameters(), lr=1e-4)
print('sample_states',type(sample_states[0]))

def reinforce(env,transform , n_episodes=1000, max_t=100, gamma=1.0, print_every=100):
    scores_deque = deque(maxlen=100)
    saved_log_probs = []
    env_info = env.reset(train_mode=True)[brain_name]
    scores = []      
    i_episode=0
    t=0
    state = env_info.vector_observations 
    for i_episode in range(1, n_episodes+1):
        saved_log_probs = []
        
        
        rewards = []
        rewarder = []
        got_reward = False
        t=0
        #print(state)
        while True: 
            t+=1
            state = torch.from_numpy(state).float().to(device)
            #print('state dot shape',state.shape)
            transform.observe(state)            
            state = transform.normalize(state)
            #print('a state=',state[0])
            action, log_prob= policy.act(state)
            #m = dist.Normal(actions_tensor, 0.8)
            #log_probs = m.log_prob(actions_tensor).sum(-1)/4.0  
            #print(actions_tensor.shape)          
            #actions_np = np.clip(actions.clone().detach().numpy(), -1, 1)                  # all actions between -1 and 1
            env_info = env.step(action.cpu().detach().numpy())[brain_name]           # send all actions to tne environment
            next_state = env_info.vector_observations         # get next state (for each agent)                     
            dones = env_info.local_done                        # see if episode finished    
            reward = env_info.rewards                         # update the score (for each agent)
            #print('reward',reward)
            #if(reward>0.0):
            #    got_reward = True
            #    print(reward)
            #    print('episode ',i_episode,'reward:',reward)
            state = next_state                               # roll over states to next time step
            saved_log_probs.append(log_prob)
            #print('log_prob',log_prob)
            
            rewards.append(reward)
            #print('rewards=:',reward)            
            if np.any(dones): 
                #print('out loop because done',t)# exit loop if episode finished
                break
            
        #print(rewards)
        total_rewards = sum([sum(i) for i in rewards])
        scores_deque.append(total_rewards )
        
        #print('rewarder',rewarder)
        scores.append(total_rewards)
        #print('scores',scores)
        #print(saved_log_probs)
        discounts = [gamma**i for i in range(len(rewards)+1)]
        #print('discounts.len',len(discounts),'rewards.len',len(rewards))
        rewards_t = list(map(list, zip(*rewards)))
        #print(rewards_t[0])
        R = [sum([a*b for a,b in zip(discounts, rewards_t_iter)]) for rewards_t_iter in rewards_t] 

        #print('R.shape',R.shape)
        #policy_loss = R*saved_log_probs
        policy_loss = []
        print(type(R))
        #save_logprob_tensor = torch.stack(saved_log_probs)
        #R_tensor = torch.tensor(R, dtype=torch.float, device=device).unsqueeze(1)
        #print('R_tensor.shape',R_tensor.shape,save_logprob_tensor.shape)
        #policy_loss = -torch.mm(save_logprob_tensor,R_tensor).sum()
        #torch.tensor(R, dtype=torch.float, device=device).unsqueeze(1)
        r1 =0        
        for r in R:
            r1 += 1
            sum_r = 0
            for lr in saved_log_probs:
                sum_r += r*lr.sum()  
            policy_loss.append(sum_r)
            
        #print('policy_loss',len(policy_loss),policy_loss)
        #for log_prob in saved_log_probs:
        #    policy_loss.append(-log_prob * R)
        all_loss = -sum(policy_loss)
        #all_loss = -sum([sum(i) for i in policy_loss]) 
        print(all_loss)
        optimizer.zero_grad()
        all_loss.backward()
        optimizer.step()
        
        if i_episode % print_every == 0:
            print('Episode {}\tAverage Score:'.format(i_episode), np.mean(scores_deque))
        if np.mean(scores_deque)>=195.0:
            print('Environment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_deque)))
            break
        
    return scores
    
scores = reinforce(env,transform )

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(scores)+1), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()